In [1]:
import numpy as np
import scipy as sp
import scipy.linalg as la 
import scipy.sparse.linalg as spla
import time 
import matplotlib.pyplot as plt

%matplotlib inline

### Problem 1

Write a function that accepts an $m x n$ matrix $A$ of rank $n$. Use Algorithm 3.1 to compute the reduced QR decomposition of A.
Consider the following tips for implementing the algorithm.

1. In Python,the operation $a = a + b$ can also be written as $a += b$.
2. Use `scipy.linalg.norm()` to compute the norm of the vector in step 6.
3. Note that steps 7 and 10 employ scalar multiplication or division, while step 9 uses vector multiplication.

To test your function, generate test cases with NumPy’s np.random module. Verify that R is upper triangular, Q is orthonormal, and QR = A. You may also want to compare your results to SciPy’s QR factorization algorithm.

In [2]:
def modified_qr(A):
    m,n = A.shape
    Q = A.copy()
    R = np.zeros((n,n))
    for i in range(n):
        R[i,i] = np.linalg.norm(Q[:,i])
        Q[:,i] = Q[:,i]/R[i,i]
        for j in range(i+1, n):
            R[i,j] = np.dot(Q[:,j,None].T,Q[:,i])
            Q[:,j] = Q[:,j] - R[i,j]*Q[:,i]
    return Q,R

In [3]:
A = np.random.random((6,4))
Q,R = la.qr(A, mode="economic")

In [4]:
Q,R

(array([[-0.71204195, -0.09324511,  0.66427465, -0.11674562],
        [-0.05869282, -0.66477546, -0.05476697, -0.311056  ],
        [-0.05960038, -0.53053881, -0.12809194,  0.66146441],
        [-0.5595976 ,  0.43117314, -0.47164466,  0.10246298],
        [-0.239043  , -0.25730815, -0.51900946, -0.53722229],
        [-0.34015904, -0.12565726, -0.21797411,  0.39111763]]),
 array([[-1.39913156, -0.89308677, -0.48216469, -1.08624546],
        [ 0.        , -1.05030867, -1.33503885, -0.21569737],
        [ 0.        ,  0.        , -0.67382144, -0.28870532],
        [ 0.        ,  0.        ,  0.        ,  0.24632446]]))

In [5]:
#verify the R is upper triangular, Q is orthonormal, and QR=A
print(np.allclose(np.triu(R), R))
print(np.allclose(np.dot(Q.T, Q), np.eye(4)))
print(np.allclose(np.dot(Q,R), A))

True
True
True


In [6]:
Q_,R_ = modified_qr(A)

In [7]:
Q_,R_

(array([[ 0.71204195,  0.09324511, -0.66427465, -0.11674562],
        [ 0.05869282,  0.66477546,  0.05476697, -0.311056  ],
        [ 0.05960038,  0.53053881,  0.12809194,  0.66146441],
        [ 0.5595976 , -0.43117314,  0.47164466,  0.10246298],
        [ 0.239043  ,  0.25730815,  0.51900946, -0.53722229],
        [ 0.34015904,  0.12565726,  0.21797411,  0.39111763]]),
 array([[ 1.39913156,  0.89308677,  0.48216469,  1.08624546],
        [ 0.        ,  1.05030867,  1.33503885,  0.21569737],
        [ 0.        ,  0.        ,  0.67382144,  0.28870532],
        [ 0.        ,  0.        ,  0.        ,  0.24632446]]))

In [8]:
#verify the R is upper triangular, Q is orthonormal, and QR=A
print(np.allclose(np.triu(R_), R_))
print(np.allclose(np.dot(Q_.T, Q_), np.eye(4)))
print(np.allclose(np.dot(Q_,R_), A))

True
True
True


### Problem 2
Write a function that accepts an invertible $nxn$ matrix A. Use the QR decomposition of $A$ to calculate $|det(A)|$.
You may use your QR decomposition algorithm from Problem 1 or SciPy’s QR routine. Can you implement this function in a single line?

In [9]:
def det(A):
    return abs(np.diag(la.qr(A)[1]).prod())

In [10]:
A = np.random.random((4,4))

In [11]:
det(A)

0.020668639745650568

In [12]:
abs(la.det(A))

0.02066863974565058

### Problem 3
Write a function that accepts an invertible $nxn$ matrix $A$ and a vector $b$ of length $n$. Use the QR decomposition to solve $Ax = b$ in the following steps:

1. Compute $Q$ and $R$.
2. Calculate $y = Q^Tb$.
3. Use back substitution to solve $Rx = y$ for $x$.

In [13]:
def qr_solve(A,b):
    #compute for Q and R
    Q,R = la.qr(A)
    #calculate y=Q.Tb
    y = np.dot(Q.T, b)
    #back substitution
    x = np.zeros(shape=(len(b),1))
    for i in range(len(x)-1,-1,-1):
        x[i] = (1./R[i,i])*(y[i] - np.sum([R[i,j]*x[j] for j in range(i+1, len(x))]))
    return x

In [14]:
A,b = np.random.random((3,3)), np.random.random((3,1))

In [15]:
qr_solve(A,b)

array([[-0.52153961],
       [ 1.40636608],
       [ 0.18477982]])

In [16]:
la.solve(A,b)

array([[-0.52153961],
       [ 1.40636608],
       [ 0.18477982]])

### Problem 4

Write a function that accepts as input a $mxn$ matrix $A$ of rank $n$. Use Algorithm 3.2 to compute the full QR decomposition of A.
Consider the following implementation details.

1.  NumPy’s `np.sign()` is an easy way to implement the `sign()` operation in step 7. However, `np.sign(0)` returns 0, which will cause a problem in the rare case that $u_0$ = 0 (which is possible if the top left entry of A is 0 to begin with). The following code defines a function that returns the sign of a single number, counting 0 as positive.
      
      `sign = lambda x: 1 if x >= 0 else -1`
       
2. In steps 9 and 10, the multiplication of $u$ and ($u_TX$) is an outer product ($xy_T$ instead of the usual $x_Ty$). Use `np.outer()` instead of `np.dot()` to handle this correctly.

In [17]:
def qr_householder(A):
    m,n = A.shape
    R = A.copy()
    Q = np.eye(m)
    sign = lambda x: 1 if x==0 else np.sign(x)
    for k in range(n):
        u = R[k:,k].copy()
        u[0] = u[0]+ sign(u[0])*np.linalg.norm(u)
        u = u/np.linalg.norm(u)
        R[k:,k:] -= 2*np.outer(u,(np.dot(u.T, R[k:,k:])))
        Q[k:,:] -= 2*np.outer(u,(np.dot(u.T, Q[k:,:])))
    return Q.T,R

In [18]:
A = np.random.random((5,3))
Q_,R_ = qr_householder(A)
Q_,R_

(array([[-0.66297693,  0.46830431,  0.48639777, -0.28345764,  0.15563303],
        [-0.00605602, -0.37892543,  0.68939075,  0.61268363,  0.07574962],
        [-0.31774128, -0.6844525 ,  0.09144749, -0.47446179, -0.44394663],
        [-0.6459451 , -0.0098265 , -0.47990861,  0.55391104, -0.21337444],
        [-0.20547584, -0.4105299 , -0.22244703, -0.11108153,  0.85289102]]),
 array([[ -1.48610068e+00,  -1.20101721e+00,  -9.12310434e-01],
        [ -3.46944695e-18,  -5.56130121e-01,  -2.83410789e-01],
        [ -1.66533454e-16,   0.00000000e+00,   8.53753969e-01],
        [ -3.33066907e-16,   0.00000000e+00,   1.11022302e-16],
        [ -1.11022302e-16,   0.00000000e+00,   5.55111512e-17]]))

In [19]:
Q,R = la.qr(A)
Q,R

(array([[-0.66297693,  0.46830431,  0.48639777, -0.28345764,  0.15563303],
        [-0.00605602, -0.37892543,  0.68939075,  0.61268363,  0.07574962],
        [-0.31774128, -0.6844525 ,  0.09144749, -0.47446179, -0.44394663],
        [-0.6459451 , -0.0098265 , -0.47990861,  0.55391104, -0.21337444],
        [-0.20547584, -0.4105299 , -0.22244703, -0.11108153,  0.85289102]]),
 array([[-1.48610068, -1.20101721, -0.91231043],
        [ 0.        , -0.55613012, -0.28341079],
        [ 0.        ,  0.        ,  0.85375397],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]))

### Problem 5 
Write a function that accepts a nonsingular $nxn$ matrix $A$. Use Algorithm 3.3 to compute its upper Hessenberg form, upper Hessenberg H and orthogonal $Q$ satisfying $A = QHQ^T$.
Test your function and compare your results to `scipy.linalg.hessenberg()`.

In [20]:
def hessenberg(A):
    m,n = A.shape
    H = A.copy()
    Q = np.eye(m)
    sign = lambda x: 1 if x==0 else np.sign(x)
    for k in range(n-2):
        u = H[k+1:,k].copy()
        u[0] = u[0] +sign(u[0])*np.linalg.norm(u)
        u = u/np.linalg.norm(u)
        H[k+1:,k:] -= 2*np.outer(u,(np.dot(u.T, H[k+1:,k:])))
        H[:,k+1:] -= 2*np.dot(np.dot(H[:,k+1:], u[:,None]),u.T[None, :])
        Q[k+1:,:] -= 2*np.outer(u, np.dot(u.T, Q[k+1:,:]))
    return H,Q.T

In [21]:
A = np.random.random((8,8))

In [22]:
H_,Q_ = hessenberg(A)

In [23]:
print(np.allclose(np.triu(H_, -1), H_))
print(np.allclose(np.dot(np.dot(Q_, H_), Q_.T), A))

True
True


In [24]:
H,Q = sp.linalg.hessenberg(A, calc_q=True)

In [25]:
print(np.allclose(np.triu(H, -1), H))
print(np.allclose(np.dot(np.dot(Q, H), Q.T), A))

True
True


Additional Exercises

In [26]:
def givens(A):
    m,n = A.shape
    R = A.copy()
    Q = np.eye(m)
    for j in range(n):
        for i in range(m-1,j,-1):
            a,b = R[i-1,j],R[i,j]
            G = np.array([[a,b],[-b,a]])/np.sqrt(a**2+b**2)
            
            R[i-1:i+1,j:] = np.dot(G, R[i-1:i+1, j:])
            Q[i-1:i+1,:] = np.dot(G, Q[i-1:i+1, :])
    return Q.T,R

In [27]:
A =np.random.random((9,9))

In [28]:
Q,R = givens(A)
print(np.allclose(np.dot(Q,R), A))

True


In [29]:
def givens_hessenberg(H):
    #H should be an upper Hessenberg form
    m,n = H.shape
    R = H.copy()
    Q = np.eye(m)
    for j in range(min(n-1, m-1)):
        i = j+1
        a,b = R[i-1,j], R[i,j]
        G = np.array([[a,b],[-b,a]])/np.sqrt(a**2+b**2)
        R[i-1:i+1,j:] = np.dot(G, R[i-1:i+1,j:])
        Q[i-1:i+1,:i+1] = np.dot(G, Q[i-1:i+1,:i+1])
    return Q.T, R

In [30]:

Q_,R_ = givens_hessenberg(H)
print(np.allclose(np.dot(Q_,R_), H))

True
